In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

try:
    loader = WebBaseLoader("https://careers.nike.com/software-engineer-ii-itc/job/R-49980")
    page_data = loader.load().pop().page_content
    print(page_data[:500])  # Print first 500 chars to check
except Exception as e:
    print(f"Error loading page: {e}")

USER_AGENT environment variable not set, consider setting it to identify your requests.























Software Engineer II, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Career


In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

try:
    chain_extract = prompt_extract | llm 
    res = chain_extract.invoke(input={'page_data': page_data})
    print("Response type:", type(res.content))
    print("Extracted JSON:", res.content)
except Exception as e:
    print(f"Error in extraction: {e}")

Response type: <class 'str'>
Extracted JSON: ```json
{
  "role": "Software Engineer II, ITC",
  "experience": "2+ years",
  "skills": [
    "Java",
    "Spring Boot framework",
    "Micro service architecture",
    "Domain driven design",
    "RESTful Services",
    "Test Driven development",
    "Junit",
    "Mockito",
    "Cucumber",
    "Monitoring and tracing tools like Splunk, SignalFx, Kibana/Grafana",
    "NoSQL data stores like Cassandra, DynamoDB, MongoDB",
    "Relational data stores like MySQL, Oracle",
    "Continuous integration, unit testing, static analysis, and automated integration tests"
  ],
  "description": "Design and deliver solutions for both digital and in-store platforms to bring best in class consumer experiences to our consumer worldwide. Contribute to the development of new features and identifying technical areas for improvement, proactively collaborating with product team members at Nike to design, develop, and deploy a highly scalable solution."
}
```


In [11]:
from langchain_core.output_parsers import JsonOutputParser

try:
    json_parser = JsonOutputParser()
    json_res = json_parser.parse(res.content)
    
    print("JSON parsed successfully!")
    print("Type:", type(json_res))
    print("Keys:", list(json_res.keys()))
    print("\nParsed data:")
    print(json_res)
    
except Exception as e:
    print(f"JSON parsing error: {e}")
    print("Raw content:", res.content[:200])

JSON parsed successfully!
Type: <class 'dict'>
Keys: ['role', 'experience', 'skills', 'description']

Parsed data:
{'role': 'Software Engineer II, ITC', 'experience': '2+ years', 'skills': ['Java', 'Spring Boot framework', 'Micro service architecture', 'Domain driven design', 'RESTful Services', 'Test Driven development', 'Junit', 'Mockito', 'Cucumber', 'Monitoring and tracing tools like Splunk, SignalFx, Kibana/Grafana', 'NoSQL data stores like Cassandra, DynamoDB, MongoDB', 'Relational data stores like MySQL, Oracle', 'Continuous integration, unit testing, static analysis, and automated integration tests'], 'description': 'Design and deliver solutions for both digital and in-store platforms to bring best in class consumer experiences to our consumer worldwide. Contribute to the development of new features and identifying technical areas for improvement, proactively collaborating with product team members at Nike to design, develop, and deploy a highly scalable solution.'}


In [13]:
type(json_res)

dict

In [15]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [17]:
import uuid
import chromadb

try:
    client = chromadb.PersistentClient('vectorstore')
    collection = client.get_or_create_collection(name="portfolio")
    
    print(f"Collection count before: {collection.count()}")
    
    if not collection.count():
        print("Adding portfolio data to ChromaDB...")
        for index, row in df.iterrows():
            collection.add(
                documents=row["Techstack"],
                metadatas={"links": row["Links"]},
                ids=[str(uuid.uuid4())]
            )
        print(f"Added {len(df)} portfolio items to ChromaDB")
    else:
        print("Collection already populated")
        
    print(f"Final collection count: {collection.count()}")
    
except Exception as e:
    print(f"ChromaDB error: {e}")

Collection count before: 0
Adding portfolio data to ChromaDB...


/Users/himanshuraj/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|


Added 20 portfolio items to ChromaDB
Final collection count: 20


In [25]:
links = collection.query(query_texts=json_res['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'l

In [29]:
job = json_res
job['skills']

['Java',
 'Spring Boot framework',
 'Micro service architecture',
 'Domain driven design',
 'RESTful Services',
 'Test Driven development',
 'Junit',
 'Mockito',
 'Cucumber',
 'Monitoring and tracing tools like Splunk, SignalFx, Kibana/Grafana',
 'NoSQL data stores like Cassandra, DynamoDB, MongoDB',
 'Relational data stores like MySQL, Oracle',
 'Continuous integration, unit testing, static analysis, and automated integration tests']

In [31]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

try:
    chain_email = prompt_email | llm
    res = chain_email.invoke({"job_description": str(job), "link_list": links})
    print("✅ Cold email generated successfully!")
    print("\n" + "="*50)
    print("GENERATED COLD EMAIL:")
    print("="*50)
    print(res.content)
except Exception as e:
    print(f"❌ Email generation error: {e}")

✅ Cold email generated successfully!

GENERATED COLD EMAIL:
Subject: Expert Software Engineering Solutions for Nike's Digital Transformation

Dear Hiring Manager,

I came across the job description for a Software Engineer II, ITC at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you design and deliver solutions for both digital and in-store platforms. With our expertise in Java, Spring Boot framework, Microservice architecture, and Domain-driven design, we can contribute to the development of new features and identify technical areas for improvement, ensuring a highly scalable solution.

Our team at AtliQ has extensive experience in developing scalable and efficient software solutions using RESTful Services, Test-Driven Development, Junit, Mockito, and Cucumber. We're also well-versed in monitoring and tracing tools like Splunk, SignalFx, and Kibana/Grafana. Our expertise in NoSQL data stores like Cassandra, DynamoDB, and MongoDB, as well as